In [ ]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

In [ ]:
#인덱스 ,index_col="patientid" 소문자로 되있네..
train = pd.read_csv("data/ml_10_medicalalert_train.csv",index_col="patientid")
test = pd.read_csv("data/ml_10_medicalalert_test.csv",index_col="patientid")
#train = pd.read_csv("data/ml_10_medicalalert_train.csv")
#test = pd.read_csv("data/ml_10_medicalalert_test.csv")

print(train.shape)
print(test.shape)


In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train.columns

In [ ]:
train.describe()

In [ ]:
obj_df = train.select_dtypes(include=["object"]).copy()
obj_df.head()

In [ ]:
#gl 결측치 심한데 gl=혈당
obj_df[obj_df.isnull().any(axis=1)].head(10)

In [ ]:
#결측치 확인.
train.info()

test.info()

In [ ]:
#상관관계 분석을 해보자.
train.corr()
#상관 관계가 안나타남으로 일딴 요인분석 차후 생각.

In [ ]:
#데이터 처리전 일딴 복사
train_c_df = train.copy()
test_c_df = train.copy()

In [ ]:
#성별 데이터 전처리
train["gender"].value_counts()

In [ ]:
train.loc[train["gender"] == "M", "gender"] = int(0)
train.loc[train["gender"] == "F", "gender"] = int(1)
train.loc[train["gender"] == "gender", "gender"] = int(0)

test.loc[test["gender"] == "M", "gender"] = int(0)
test.loc[test["gender"] == "F", "gender"] = int(1)

train["gender"].value_counts()

In [ ]:
##데이터 타입 못봐서 ... //마지막 오답노트 age->int
#위에 쓴거 결측치도 없는데 데이터 타입 안바뀌는 건 이해할 수가 살짝 없음.
#train["gender"] = train["gender"].astype(int)
train["gender"] = pd.to_numeric(train["gender"], errors="coerce")
test["gender"] = pd.to_numeric(test["gender"], errors="coerce")


train["age"] = pd.to_numeric(train["age"], errors="coerce")
train["di"] = pd.to_numeric(train["di"], errors="coerce")
train["copd"] = pd.to_numeric(train["copd"], errors="coerce")
train["chf"] = pd.to_numeric(train["chf"], errors="coerce")
train["ht"] = pd.to_numeric(train["ht"], errors="coerce")
train["afib"] = pd.to_numeric(train["afib"], errors="coerce")
train["w"] = pd.to_numeric(train["w"], errors="coerce")
train["pw"] = pd.to_numeric(train["pw"], errors="coerce")
      


train.info()

In [ ]:
train.loc[train["alert"] == "No", "alert"] = 0
train.loc[train["alert"] == "Yes", "alert"] = 1
train.loc[train["alert"] == "alert", "alert"] = 1

In [ ]:
train.info()
test.info()

In [ ]:
#안돌려지는건 처음인데 ... 상관관계가 없을꺼라고 예상
train.corr()

In [ ]:
#gl은 너무 데이터가 없어서 컬럼을 제거해야겠고 spo2도 마찬가지 
#bps부터는 데이터의 상의함이 생겨서 정확도를 위해선 보간 처리를 해야 될꺼같지만 일딴 시간문제상 보류
#일딴 di,chf는 연속되는 실수형이 아님으로 결측치는 0으로 할당



train.dropna(axis=1)
test.dropna(axis=1)

train["age"] = train["age"].fillna(0)
train["di"] = train["di"].fillna(0)



train["chf"] = train["chf"].fillna(0)
train["age"] = train["age"].fillna(0)




train["gender"].fillna(0, inplace=True)


In [ ]:
y_train = train["alert"]
X_train = train.drop("alert",axis=1)
X_test = test

X_train.columns

cat_feature = ["timestamp","bps","di","copd","chf","ht","afib","w","pw"]

for cat_name in cat_feature:
    dummy = pd.get_dummies(train[cat_name], prefix=cat_name)
    X_train = pd.concat([X_train,dummy], axis=1)
    X_train.drop(cat_name,axis=1,inplace=True)
    
for cat_name in cat_feature:
    dummy = pd.get_dummies(test[cat_name], prefix=cat_name)
    X_test = pd.concat([X_test,dummy], axis=1)
    X_test.drop(cat_name,axis=1,inplace=True)
    
X_train.shape

X_test.shape

set(X_train.columns) - set(X_test.columns)


X_train.shape

X_test.shape

In [ ]:
X_train.head()



In [ ]:
y_train.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
#의사 결정나무
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(test)

sub["alert"] = pred
sub.to_csv("./saveFlie.csv", index = False)